# PrefixSpan

In [1]:
pip install prefixspan

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for prefixspan: filename=prefixspan-0.5.2-py3-none-any.whl size=11220 sha256=13c1c9445aa4a383d619cde718f43beb7e9440ba83214cdc7cddd85a07377b1f
  Stored in directory: /home/onyxia/.cache/pip/wheels/bf/96/ee/9e087a6d0d3163ee363c069bf80eaa4ca4f5ee51f2b2b0521c
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=33a20691732fe80fe52d4272690c58c187e2634b55b697d7f45fe93a7944ce46
  Stored in directory: /home/onyxia/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for extratools: filename=extratools-0.8.2.1-py3-none-any.whl size=28868 sha256=716987e1ab6940e5b134cce0d3334de7845c6411f1ebb844978d8362535e80df
  Stored in directory: /home/onyxia/.cache/pip/wheels/70/f3/03/3a98db17111f679c3291413b81d2a1e6e1bad5a3441175ace7
Successfully built prefixspan docopt extratools
Note: you may n

In [2]:
#Import des librairies

from prefixspan import PrefixSpan

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.express as px

In [5]:
%%capture capt
p_soins=pd.read_csv("../data/p_soins.csv", index_col=0)
p_soins_IC=pd.read_csv("../data/p_soins_IC.csv", index_col=0)

In [11]:
p_soins.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,P0,4,05K051,05M042,05M16T,05M09T,05M092,05C191,05M20Z,05M20Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P6,4,02C05J,05M093,04M132,05C222,23M103,04M053,04M24E,Décès,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P8,2,11M041,06C194,Décès,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P13,2,05M04T,05K051,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P16,4,08C11V,23M15Z,05K102,05M08T,05K102,05C191,23M101,05K191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def truncateGHM(df):
    for i in tqdm(range(2, df.shape[1])):
        df[str(i)]=df[str(i)].map(lambda x: str(x)[:5])
    df = df.replace('nan', np.nan)

In [13]:
truncateGHM(p_soins)
p_soins=p_soins.replace('nan', np.nan)

truncateGHM(p_soins_IC)
p_soins_IC=p_soins_IC.replace('nan', np.nan)

100%|██████████| 123/123 [00:00<00:00, 631.89it/s]


In [14]:
p_soins.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,P0,4,05K05,05M04,05M16,05M09,05M09,05C19,05M20,05M20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P6,4,02C05,05M09,04M13,05C22,23M10,04M05,04M24,Décès,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P8,2,11M04,06C19,Décès,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P13,2,05M04,05K05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P16,4,08C11,23M15,05K10,05M08,05K10,05C19,23M10,05K19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
def motifs_frequents(topk):
    top_freq=[]
    top_effectif=[]
    top_motif=[]
    results=pd.DataFrame()

    for length in range(1,5):
        corpus = p_soins.drop(["0", "1"], axis=1).stack().groupby(level=0).apply(list).tolist()
        ps = PrefixSpan(corpus)
        ps.minlen=length
        if ps.topk(k=topk) != [] :
            effectif = len(corpus)
            top_effectif.append(ps.topk(k=topk)[topk-1][0])
            top_freq.append(round(ps.topk(k=topk)[topk-1][0]/effectif,3))
            top_motif.append(ps.topk(k=topk)[topk-1][1])
        else:
            top_freq.append(0)
            top_effectif.append(0)
            top_motif.append([])

        results[f"len{length}_effectif"]=top_effectif
        results[f"len{length}_freq"]=top_freq
        results[f"len{length}_motif"]=top_motif
        top_freq=[]
        top_motif=[]
        top_effectif=[]

    return results

## Motifs de GHM fréquents sur l'ensemble de la population

### Motif le plus fréquent

In [44]:
freq1=motifs_frequents(1)
freq1

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,6618,0.658,[Décès],3298,0.328,"[05M09, Décès]",1206,0.12,"[05M09, 05M09, Décès]",560,0.056,"[05M09, 05M09, 05M09, Décès]"


### Second motif le plus fréquent

In [45]:
freq2=motifs_frequents(2)
freq2

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,4637,0.461,[05M09],1632,0.162,"[05M09, 05M09]",728,0.072,"[05M09, 05M09, 05M09]",367,0.037,"[05M09, 05M09, 05M09, 05M09]"


### Troisième motif le plus fréquent

In [46]:
freq3=motifs_frequents(3)
freq3

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,3298,0.328,"[05M09, Décès]",1331,0.132,"[04M05, Décès]",560,0.056,"[05M09, 05M09, 05M09, Décès]",277,0.028,"[05M09, 05M09, 05M09, 05M09, Décès]"


In [49]:
print(pd.concat([freq1, freq2, freq3]).iloc[:,:-3].to_latex())

\begin{tabular}{lrrlrrlrrl}
\toprule
 & len1_effectif & len1_freq & len1_motif & len2_effectif & len2_freq & len2_motif & len3_effectif & len3_freq & len3_motif \\
\midrule
0 & 6618 & 0.658000 & ['Décès'] & 3298 & 0.328000 & ['05M09', 'Décès'] & 1206 & 0.120000 & ['05M09', '05M09', 'Décès'] \\
0 & 4637 & 0.461000 & ['05M09'] & 1632 & 0.162000 & ['05M09', '05M09'] & 728 & 0.072000 & ['05M09', '05M09', '05M09'] \\
0 & 3298 & 0.328000 & ['05M09', 'Décès'] & 1331 & 0.132000 & ['04M05', 'Décès'] & 560 & 0.056000 & ['05M09', '05M09', '05M09', 'Décès'] \\
\bottomrule
\end{tabular}



In [32]:
def motifs_frequents_cluster(topk):
    top_freq=[]
    top_effectif=[]
    top_motif=[]
    results=pd.DataFrame()

    for length in range(1,5):
        for cluster in range(1, 6):
            corpus = p_soins[p_soins['1']==cluster].drop(["0", "1"], axis=1).stack().groupby(level=0).apply(list).tolist()
            ps = PrefixSpan(corpus)
            ps.minlen=length
            if ps.topk(k=topk) != [] :
                effectif_cluster = len(corpus)
                top_effectif.append(ps.topk(k=topk)[topk-1][0])
                top_freq.append(round(ps.topk(k=topk)[topk-1][0]/effectif_cluster,3))
                top_motif.append(ps.topk(k=topk)[topk-1][1])
            else:
                top_freq.append(0)
                top_effectif.append(0)
                top_motif.append([])

        results[f"len{length}_effectif"]=top_effectif
        results[f"len{length}_freq"]=top_freq
        results[f"len{length}_motif"]=top_motif
        top_freq=[]
        top_motif=[]
        top_effectif=[]

    return results

## Motifs de GHM fréquents par clusters de patients

### Motif le plus fréquent

In [33]:
top1=motifs_frequents_cluster(1)
top1

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,833,0.610,[Décès],507,0.371,"[05M09, Décès]",301,0.220,"[05M09, 05M09, Décès]",181,0.133,"[05M09, 05M09, 05M09, Décès]"
1,3467,0.685,[Décès],1542,0.305,"[05M09, Décès]",336,0.066,"[05M09, 05M09, Décès]",91,0.018,"[05M09, 05M09, 05M09, Décès]"
2,22,0.629,[23M20],15,0.429,"[23M20, 23M20]",8,0.229,"[23M20, 23M20, 23M20]",8,0.229,"[07M14, 07M14, 07M14, 07M14]"
3,2082,0.651,[Décès],1111,0.347,"[05M09, Décès]",487,0.152,"[05M09, 05M09, Décès]",234,0.073,"[05M09, 05M09, 05M09, Décès]"
4,224,0.579,[05M09],132,0.341,"[05M09, 05M09]",75,0.194,"[05M09, 05M09, 05M09]",52,0.134,"[05M09, 05M09, 05M09, 05M09]"


In [34]:
print(top1.iloc[:,:-3].to_latex())

\begin{tabular}{lrrlrrlrrl}
\toprule
 & len1_effectif & len1_freq & len1_motif & len2_effectif & len2_freq & len2_motif & len3_effectif & len3_freq & len3_motif \\
\midrule
0 & 833 & 0.610000 & ['Décès'] & 507 & 0.371000 & ['05M09', 'Décès'] & 301 & 0.220000 & ['05M09', '05M09', 'Décès'] \\
1 & 3467 & 0.685000 & ['Décès'] & 1542 & 0.305000 & ['05M09', 'Décès'] & 336 & 0.066000 & ['05M09', '05M09', 'Décès'] \\
2 & 22 & 0.629000 & ['23M20'] & 15 & 0.429000 & ['23M20', '23M20'] & 8 & 0.229000 & ['23M20', '23M20', '23M20'] \\
3 & 2082 & 0.651000 & ['Décès'] & 1111 & 0.347000 & ['05M09', 'Décès'] & 487 & 0.152000 & ['05M09', '05M09', 'Décès'] \\
4 & 224 & 0.579000 & ['05M09'] & 132 & 0.341000 & ['05M09', '05M09'] & 75 & 0.194000 & ['05M09', '05M09', '05M09'] \\
\bottomrule
\end{tabular}



### Second motif le plus fréquent

In [35]:
top2=motifs_frequents_cluster(2)
top2

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,777,0.569,[05M09],431,0.316,"[05M09, 05M09]",256,0.187,"[05M09, 05M09, 05M09]",155,0.113,"[05M09, 05M09, 05M09, 05M09]"
1,2032,0.401,[05M09],489,0.097,"[04M05, Décès]",104,0.021,"[05M09, 05M09, 05M09]",25,0.005,"[05M09, 05M09, 05M09, 05M09]"
2,18,0.514,[05M09],10,0.286,"[23M20, 16M11]",8,0.229,"[23M20, 23M20, 23M20]",7,0.200,"[07M14, 07M14, 07M14, 07M14, 07M14]"
3,1586,0.496,[05M09],642,0.201,"[05M09, 05M09]",285,0.089,"[05M09, 05M09, 05M09]",129,0.040,"[05M09, 05M09, 05M09, 05M09]"
4,223,0.576,[Décès],129,0.333,"[05M09, Décès]",75,0.194,"[05M09, 05M09, Décès]",48,0.124,"[05M09, 05M09, 05M09, Décès]"


In [36]:
print(top2.iloc[:,:-3].to_latex())

\begin{tabular}{lrrlrrlrrl}
\toprule
 & len1_effectif & len1_freq & len1_motif & len2_effectif & len2_freq & len2_motif & len3_effectif & len3_freq & len3_motif \\
\midrule
0 & 777 & 0.569000 & ['05M09'] & 431 & 0.316000 & ['05M09', '05M09'] & 256 & 0.187000 & ['05M09', '05M09', '05M09'] \\
1 & 2032 & 0.401000 & ['05M09'] & 489 & 0.097000 & ['04M05', 'Décès'] & 104 & 0.021000 & ['05M09', '05M09', '05M09'] \\
2 & 18 & 0.514000 & ['05M09'] & 10 & 0.286000 & ['23M20', '16M11'] & 8 & 0.229000 & ['23M20', '23M20', '23M20'] \\
3 & 1586 & 0.496000 & ['05M09'] & 642 & 0.201000 & ['05M09', '05M09'] & 285 & 0.089000 & ['05M09', '05M09', '05M09'] \\
4 & 223 & 0.576000 & ['Décès'] & 129 & 0.333000 & ['05M09', 'Décès'] & 75 & 0.194000 & ['05M09', '05M09', 'Décès'] \\
\bottomrule
\end{tabular}



### Troisième motif le plus fréquent

In [39]:
top3=motifs_frequents_cluster(3)
top3

,len1_effectif,len1_freq,len1_motif,len2_effectif,len2_freq,len2_motif,len3_effectif,len3_freq,len3_motif,len4_effectif,len4_freq,len4_motif
0,507,0.371,"[05M09, Décès]",301,0.220,"[05M09, 05M09, Décès]",181,0.133,"[05M09, 05M09, 05M09, Décès]",109,0.080,"[05M09, 05M09, 05M09, 05M09, Décès]"
1,1542,0.305,"[05M09, Décès]",418,0.083,"[05M09, 05M09]",102,0.020,"[02C05, 05M09, Décès]",21,0.004,"[05M09, 05M09, 05M09, 05M09, Décès]"
2,15,0.429,"[23M20, 23M20]",9,0.257,"[23M20, 05M09]",8,0.229,"[23M20, 23M20, 23M20]",7,0.200,"[07M14, 07M14, 07M14, 07M14, 07M14, 07M14]"
3,1111,0.347,"[05M09, Décès]",505,0.158,"[04M05, Décès]",234,0.073,"[05M09, 05M09, 05M09, Décès]",109,0.034,"[05M09, 05M09, 05M09, 05M09, Décès]"
4,167,0.432,[05K10],88,0.227,"[05K10, 05M09]",56,0.145,"[05K10, 05M09, 05M09]",35,0.090,"[05M09, 05M09, 05M09, 05M09, 05M09]"


In [40]:
print(top3.iloc[:,:-3].to_latex())

\begin{tabular}{lrrlrrlrrl}
\toprule
 & len1_effectif & len1_freq & len1_motif & len2_effectif & len2_freq & len2_motif & len3_effectif & len3_freq & len3_motif \\
\midrule
0 & 507 & 0.371000 & ['05M09', 'Décès'] & 301 & 0.220000 & ['05M09', '05M09', 'Décès'] & 181 & 0.133000 & ['05M09', '05M09', '05M09', 'Décès'] \\
1 & 1542 & 0.305000 & ['05M09', 'Décès'] & 418 & 0.083000 & ['05M09', '05M09'] & 102 & 0.020000 & ['02C05', '05M09', 'Décès'] \\
2 & 15 & 0.429000 & ['23M20', '23M20'] & 9 & 0.257000 & ['23M20', '05M09'] & 8 & 0.229000 & ['23M20', '23M20', '23M20'] \\
3 & 1111 & 0.347000 & ['05M09', 'Décès'] & 505 & 0.158000 & ['04M05', 'Décès'] & 234 & 0.073000 & ['05M09', '05M09', '05M09', 'Décès'] \\
4 & 167 & 0.432000 & ['05K10'] & 88 & 0.227000 & ['05K10', '05M09'] & 56 & 0.145000 & ['05K10', '05M09', '05M09'] \\
\bottomrule
\end{tabular}

